In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [42]:
degs = qs::qread('_targets/objects/deg_ensmus_MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none')
degs %>% head

,GeneID,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,tag,ensmusg,gsea_sort_score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,Cdh20,3.307989e-33,0.5443095,0.940,0.846,5.771448e-29,MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none,ENSMUSG00000050840,15.370600
2,Spock3,1.678772e-27,0.5064675,0.855,0.746,2.928953e-23,MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none,ENSMUSG00000054162,11.412377
3,Usp31,4.659305e-21,0.3688095,0.817,0.677,8.129089e-17,MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none,ENSMUSG00000063317,5.934129
4,Shtn1,8.326713e-23,0.3276810,0.936,0.864,1.452762e-18,MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none,ENSMUSG00000041362,5.845110
5,Rnf220,2.263840e-22,0.3274835,0.978,0.952,3.949722e-18,MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none,ENSMUSG00000028677,5.699337
6,Nkain2,1.405446e-25,0.1813910,1.000,1.000,2.452081e-21,MOL___all.obob5v5__v__all.obobBL6___restored_grouping.pos_restored_vs_none,ENSMUSG00000069670,3.738553


In [63]:
make_gost = function(degs){
    degs = degs %>% filter(p_val_adj < 0.05)
    gene_list_all = degs %>% pull(ensmusg)
    gene_list_pos = degs %>% filter(avg_log2FC > 0) %>% pull(ensmusg)
    gene_list_neg = degs %>% filter(avg_log2FC < 0) %>% arrange(gsea_sort_score) %>% pull(ensmusg)
    gene_list = list(all = gene_list_all,
                     up = gene_list_pos,
                     down = gene_list_neg)
    gost_results = gprofiler2::gost(query = gene_list,
                                    organism = "mmusculus",
                                    ordered_query = TRUE,
                                    exclude_iea = FALSE, 
                                    evcodes = TRUE)
    gost_results$result = gost_results$result %>%
    rowwise %>%
    mutate(gene_ids = str_split(intersection, fixed(','))) %>%
    mutate(gene_ids = {degs %>% 
                       filter(ensmusg %in% gene_ids) %>% 
                       pull(GeneID) %>% 
                       paste0(collapse=',')}) %>%
    ungroup
    gost_results
}

In [44]:
gost_results$result %>% arrange(p_value) %>% filter(term_size < 500) %>% filter(query == 'all')

query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents,evidence_codes,intersection
<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>,<chr>,<chr>
all,TRUE,5.889157e-05,481,119,13,0.10924370,0.02702703,GO:0098978,GO:CC,glutamatergic synapse,27145,3691,GO:0045202,"IDA IMP IEP,IDA,IDA IMP,ISO,ISO IEA,ISO IEA,IDA IMP ISO IEA,ISO,IDA IMP,EXP IDA ISO,ISO IEA,IDA IMP,EXP IDA ISO","ENSMUSG00000032537,ENSMUSG00000036510,ENSMUSG00000034912,ENSMUSG00000020458,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000028399,ENSMUSG00000040003,ENSMUSG00000021314,ENSMUSG00000054693,ENSMUSG00000047976,ENSMUSG00000017412,ENSMUSG00000056755"
all,TRUE,7.118440e-05,450,129,13,0.10077519,0.02888889,GO:0097060,GO:CC,synaptic membrane,27145,3240,"GO:0045202, GO:0098590","IDA IEP,IDA,ISO,ISO IEA,IDA,ISO,NAS IEA,IDA ISO IBA IEA,ISS ISO IEA,ISS ISO IEA,IEA,IDA ISO TAS,ISO IEA","ENSMUSG00000060534,ENSMUSG00000036510,ENSMUSG00000020458,ENSMUSG00000040265,ENSMUSG00000028399,ENSMUSG00000040003,ENSMUSG00000027799,ENSMUSG00000054693,ENSMUSG00000047976,ENSMUSG00000069601,ENSMUSG00000023267,ENSMUSG00000056755,ENSMUSG00000030310"
all,TRUE,1.010030e-04,316,129,11,0.08527132,0.03481013,GO:0045211,GO:CC,postsynaptic membrane,27145,2442,"GO:0097060, GO:0098794","IDA IEP,ISO,ISO IEA,ISO,NAS IEA,ISO,ISO IEA,ISS ISO IEA,IEA,ISO,ISO IEA","ENSMUSG00000060534,ENSMUSG00000020458,ENSMUSG00000040265,ENSMUSG00000040003,ENSMUSG00000027799,ENSMUSG00000054693,ENSMUSG00000047976,ENSMUSG00000069601,ENSMUSG00000023267,ENSMUSG00000056755,ENSMUSG00000030310"
all,TRUE,3.391449e-04,436,55,9,0.16363636,0.02064220,GO:0034329,GO:BP,cell junction assembly,27350,9835,"GO:0022607, GO:0034330","IBA,IMP,IMP,IBA,IDA IMP,ISO,ISO,IDA IMP ISO IEA,IDA","ENSMUSG00000050840,ENSMUSG00000032854,ENSMUSG00000032537,ENSMUSG00000036510,ENSMUSG00000034912,ENSMUSG00000022425,ENSMUSG00000040265,ENSMUSG00000028399,ENSMUSG00000040003"
all,TRUE,3.663280e-04,253,68,8,0.11764706,0.03162055,GO:0050807,GO:BP,regulation of synapse organization,27350,15316,"GO:0050803, GO:0050808, GO:0051128","IMP,IDA IMP,IDA IMP,ISO IEA,ISS ISO IBA IEA,IDA IMP ISO IEA,ISO,ISO","ENSMUSG00000032537,ENSMUSG00000036510,ENSMUSG00000034912,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000028399,ENSMUSG00000040003,ENSMUSG00000054693"
all,TRUE,4.508854e-04,260,68,8,0.11764706,0.03076923,GO:0050803,GO:BP,regulation of synapse structure or activity,27350,15312,GO:0065008,"IMP,IDA IMP,IDA IMP,ISO IEA,ISS ISO IBA IEA,IDA IMP ISO IEA,ISO,ISO","ENSMUSG00000032537,ENSMUSG00000036510,ENSMUSG00000034912,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000028399,ENSMUSG00000040003,ENSMUSG00000054693"
all,TRUE,9.459126e-04,484,130,13,0.10000000,0.02685950,GO:0050808,GO:BP,synapse organization,27350,15317,GO:0034330,"IMP,IDA IMP,IDA IMP,ISO IEA,ISS ISO IBA IEA,IDA IMP ISO IBA IEA,IDA ISO IEA,EXP IDA ISO,IMP ISS,IMP IBA,IMP IBA,IMP,IDA IBA","ENSMUSG00000032537,ENSMUSG00000036510,ENSMUSG00000034912,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000028399,ENSMUSG00000040003,ENSMUSG00000054693,ENSMUSG00000069601,ENSMUSG00000035357,ENSMUSG00000017412,ENSMUSG00000030310,ENSMUSG00000032036"
all,TRUE,1.016249e-03,278,96,9,0.09375000,0.03237410,GO:0098742,GO:BP,cell-cell adhesion via plasma-membrane adhesion molecules,27350,22041,GO:0098609,"IBA IEA,IBA IEA,IEA,IDA IMP,IDA ISO IBA IEA,ISO,IEA,IBA IEA,ISS","ENSMUSG00000050840,ENSMUSG00000036510,ENSMUSG00000022883,ENSMUSG00000034912,ENSMUSG00000028399,ENSMUSG00000040003,ENSMUSG00000055421,ENSMUSG00000047216,ENSMUSG00000047109"
all,TRUE,3.366132e-03,9,7,2,0.28571429,0.22222222,GO:0038007,GO:BP,netrin-activated signaling pathway,27350,11120,GO:0007166,"ISS ISO,IEA","ENSMUSG00000041362,ENSMUSG00000060534"


In [62]:
gost_results$result = gost_results$result %>%
rowwise %>%
mutate(gene_ids = str_split(intersection, fixed(','))) %>%
mutate(gene_ids = {degs %>% 
                   filter(ensmusg %in% gene_ids) %>% 
                   pull(GeneID) %>% 
                   paste0(collapse=',')}) %>%
ungroup

In [58]:
filter(degs, ensmusg %in% c("ENSMUSG00000020458" , "ENSMUSG00000040265", "ENSMUSG00000040003")) %>% pull(GeneID) %>% paste0(collapse=',')

[1] "Rtn4,Dnm3,Magi2"

In [60]:
gost_results$result %>% head

query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents,evidence_codes,intersection,gene_ids
<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>,<chr>,<chr>,<chr>
all,TRUE,1.852740e-06,1605,55,18,0.3272727,0.01121495,GO:0048699,GO:BP,generation of neurons,27350,14979,GO:0022008,"IMP IGI ISS ISO IEA,IMP,IMP ISO IBA IEA,IMP,IMP,IEA,IDA IMP ISO IBA IEA,IMP,IEA,IDA,IMP ISS ISO IEA,IMP IBA,ISO IEA,ISS ISO IBA IEA,IDA,IDA IMP,ISO IEA,ISS ISO","ENSMUSG00000041362,ENSMUSG00000028677,ENSMUSG00000060534,ENSMUSG00000032854,ENSMUSG00000058070,ENSMUSG00000022762,ENSMUSG00000032537,ENSMUSG00000038732,ENSMUSG00000022883,ENSMUSG00000034912,ENSMUSG00000020458,ENSMUSG00000038718,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000032228,ENSMUSG00000028399,ENSMUSG00000060012,ENSMUSG00000040003","Shtn1,Rnf220,Dcc,Ugt8a,Eml1,Ncam2,Ephb1,Mboat1,Robo1,Mdga2,Rtn4,Pbx3,Dnm3,Tanc2,Tcf12,Ptprd,Kif13b,Magi2"
all,TRUE,2.007116e-06,1844,150,33,0.2200000,0.01789588,GO:0022008,GO:BP,neurogenesis,27350,7539,"GO:0007399, GO:0030154","IMP IGI ISS ISO IEA,IMP,IMP ISO IBA IEA,IMP,IMP,IEA,IDA IMP ISO IBA IEA,IMP,IEA,IDA,ISO,IMP ISS ISO IEA,IMP IBA,ISO IEA,ISS ISO IBA IEA,IDA,IDA IMP,ISO IEA,ISS ISO,ISO,IMP,IEA,IMP ISO,IMP,IEA,ISS ISO IEA,IMP,IMP,IMP IGI IBA,IMP IGI,IEA,ISO,ISO IBA","ENSMUSG00000041362,ENSMUSG00000028677,ENSMUSG00000060534,ENSMUSG00000032854,ENSMUSG00000058070,ENSMUSG00000022762,ENSMUSG00000032537,ENSMUSG00000038732,ENSMUSG00000022883,ENSMUSG00000034912,ENSMUSG00000022425,ENSMUSG00000020458,ENSMUSG00000038718,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000032228,ENSMUSG00000028399,ENSMUSG00000060012,ENSMUSG00000040003,ENSMUSG00000054693,ENSMUSG00000047976,ENSMUSG00000032517,ENSMUSG00000069601,ENSMUSG00000004207,ENSMUSG00000052512,ENSMUSG00000056755,ENSMUSG00000021097,ENSMUSG00000032036,ENSMUSG00000022995,ENSMUSG00000048402,ENSMUSG00000056222,ENSMUSG00000071454,ENSMUSG00000019970","Shtn1,Rnf220,Dcc,Ugt8a,Eml1,Ncam2,Ephb1,Mboat1,Robo1,Mdga2,Enpp2,Rtn4,Pbx3,Dnm3,Tanc2,Tcf12,Ptprd,Kif13b,Magi2,Adam10,Kcna1,Mobp,Ank3,Psap,Nav2,Grm7,Clmn,Kirrel3,Enah,Gli2,Spock1,Dtnb,Sgk1"
all,TRUE,2.264915e-06,1088,53,15,0.2830189,0.01378676,GO:0000902,GO:BP,cell morphogenesis,27350,307,GO:0009653,"IBA,IMP ISS ISO,IMP ISO IBA IEA,IMP,IBA,IDA IMP IBA,IBA,IEA,ISS ISO IEA,ISO IEA,IMP ISS ISO IEA,ISO IEA,ISS ISO IBA IEA,IDA IMP,ISO IEA","ENSMUSG00000050840,ENSMUSG00000041362,ENSMUSG00000060534,ENSMUSG00000032854,ENSMUSG00000026109,ENSMUSG00000032537,ENSMUSG00000036510,ENSMUSG00000022883,ENSMUSG00000040990,ENSMUSG00000022425,ENSMUSG00000020458,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000028399,ENSMUSG00000060012","Cdh20,Shtn1,Dcc,Ugt8a,Tmeff2,Ephb1,Cdh8,Robo1,Sh3kbp1,Enpp2,Rtn4,Dnm3,Tanc2,Ptprd,Kif13b"
all,TRUE,4.129645e-06,3423,134,43,0.3208955,0.01256208,GO:0051641,GO:BP,cellular localization,27350,15980,"GO:0009987, GO:0051179","IMP,ISS ISO IEA,IMP ISO IEA,IBA,IDA IMP IGI ISO IBA IEA,ISS ISO IEA,ISS ISO IBA IEA,IBA,IBA,ISS ISO IEA,IDA ISS ISO IBA,IDA ISS IBA,IMP,IMP ISS ISO IBA IEA,ISO IBA IEA,ISO IBA IEA,IDA IMP IBA,EXP IDA NAS,IMP,IDA ISS IBA IEA,IDA,IMP,ISO,ISO IEA,IEA,ISO,ISO IEA,IDA IMP ISS ISO IEA,IEA,IMP ISS ISO IEA,IBA,ISO IEA,IDA IMP ISO IEA,ISO IEA,EXP IDA,IEA,IBA,IMP,IPI,ISO IEA,IEA,ISO IEA,IGI ISA IBA IEA","ENSMUSG00000032854,ENSMUSG00000057614,ENSMUSG00000020458,ENSMUSG00000024277,ENSMUSG00000040265,ENSMUSG00000053580,ENSMUSG00000020658,ENSMUSG00000034265,ENSMUSG00000021969,ENSMUSG00000032841,ENSMUSG00000040003,ENSMUSG00000027799,ENSMUSG00000038894,ENSMUSG00000068747,ENSMUSG00000021143,ENSMUSG00000028152,ENSMUSG00000021314,ENSMUSG00000054693,ENSMUSG00000024598,ENSMUSG00000035441,ENSMUSG00000037996,ENSMUSG00000007476,ENSMUSG00000074111,ENSMUSG00000047976,ENSMUSG00000024535,ENSMUSG00000052303,ENSMUSG00000031555,ENSMUSG00000069601,ENSMUSG00000056515,ENSMUSG00000004207,ENSMUSG00000002910,ENSMUSG00000023923,ENSMUSG00000017412,ENSMUS

In [38]:
degs %>% filter(avg_log2FC < 0) %>% arrange(gsea_sort_score)

GeneID,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,tag,ensmusg,gsea_sort_score
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
Eml1,3.588216e-19,-0.5443518,0.318,0.510,6.260360e-15,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000058070,-7.7316471
Pcdh7,6.187001e-12,-0.8257245,0.242,0.374,1.079446e-07,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000029108,-5.7526564
9330111N05Rik,5.008116e-08,-1.3383506,0.263,0.353,8.737660e-04,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000097098,-4.0934858
Man1a,4.775082e-10,-0.7036336,0.178,0.291,8.331086e-06,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000003746,-3.5739652
Lrmda,2.153696e-08,-0.7655975,0.204,0.301,3.757554e-04,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000063458,-2.6222441
Tmeff2,2.807004e-13,-0.2672879,1.000,1.000,4.897380e-09,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000026109,-2.2211721
Rnf220,3.611123e-11,-0.3370544,0.934,0.952,6.300326e-07,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000028677,-2.0899523
Tnik,1.893460e-09,-0.4438682,0.343,0.461,3.303519e-05,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000027692,-1.9889837
Pcdh9,7.669273e-14,-0.2129435,1.000,1.000,1.338058e-09,MOL___all.obob5v5__v__all.obobBL6___fgf1_grouping.neg_vs_none,ENSMUSG00000055421,-1.8895597
